<a href="https://colab.research.google.com/github/SHASHIKATSHEGOKAR/Recommendation_System-/blob/main/netflix_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Librarys**

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import linear_kernel

**Loading Dataset and some EDA**

In [ ]:
df = pd.read_csv('netflix.csv')
df.head()

,imdb_id,title,popular_rank,certificate,startYear,endYear,episodes,runtime,type,orign_country,language,plot,summary,rating,numVotes,genres,isAdult,cast,image_url
0,tt4052886,Lucifer,1,15,2016.0,NaN,93.0,42,tvSeries,United States,English,Lucifer Morningstar has decided he's had enoug...,"Lucifer Morningstar, bored from his sulking li...",8.1,250884.0,"Crime,Drama,Fantasy",0,"['Tom Ellis', 'Lauren German', 'Lesley-Ann Bra...",https://m.media-amazon.com/images/M/MV5BNzY1Yj...
1,tt0993840,Army of the Dead,2,18,2021.0,NaN,NaN,148,movie,United States,English,"Following a zombie outbreak in Las Vegas, a gr...","With the abandoned, walled city of Las Vegas o...",5.8,110780.0,"Action,Crime,Horror",0,"['Dave Bautista', 'Ella Purnell', 'Ana de la R...",https://m.media-amazon.com/images/M/MV5BNGY0Nz...
2,tt7255502,The Kominsky Method,3,18,2018.0,2021.0,22.0,30,tvSeries,United States,English,"An aging actor, who long ago enjoyed a brush w...",Michael Douglas plays an actor who made it big...,8.2,28795.0,"Comedy,Drama",0,"['Michael Douglas', 'Sarah Baker', 'Graham Rog...",https://m.media-amazon.com/images/M/MV5BMzA0YT...
3,tt0108778,Friends,4,13+,1994.0,2004.0,235.0,22,tvSeries,United States,English,Follows the personal and professional lives of...,"Ross Geller, Rachel Green, Monica Geller, Joey...",8.9,861843.0,"Comedy,Romance",0,"['Jennifer Aniston', 'Courteney Cox', 'Lisa Ku...",https://m.media-amazon.com/images/M/MV5BNDVkYj...
4,tt9251798,Ragnarok,5,18,2020.0,NaN,12.0,45,tvSeries,Norway,Norwegian,A small Norwegian town experiencing warm winte...,In the small fictional town of Edda coming of ...,7.5,26606.0,"Action,Drama,Fantasy",0,"['David Stakston', 'Jonas Strand Gravli', 'Her...",https://m.media-amazon.com/images/M/MV5BODM3NT...


In [ ]:
df.mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


startYear     2015.606653
endYear       2016.613677
episodes        38.510851
rating           6.696400
numVotes     19617.784833
isAdult          0.000000
dtype: float64

In [ ]:
df.describe(include='O')

,imdb_id,title,popular_rank,certificate,runtime,type,orign_country,language,plot,summary,genres,cast,image_url
count,7008,7008,7008,2409,7006,7006,7008,7008,6609,7008,7006,7008,7008
unique,7008,7008,7008,24,260,10,82,71,6331,6332,500,6281,6177
top,tt4052886,Lucifer,1,18,\N,movie,United States,English,-,-,Comedy,-,https://m.media-amazon.com/images/G/01/imdb/im...
freq,1,1,1,625,1582,2923,2836,3834,277,676,713,604,826


In [ ]:
df.describe()

,startYear,endYear,episodes,rating,numVotes,isAdult
count,6493.000000,1126.000000,2304.000000,6000.000000,6.000000e+03,7008.0
mean,2015.606653,2016.613677,38.510851,6.696400,1.961778e+04,0.0
std,7.855886,5.195806,237.894838,1.209824,7.975237e+04,0.0
min,1932.000000,1969.000000,1.000000,1.700000,5.000000e+00,0.0
25%,2015.000000,2016.000000,8.000000,6.000000,3.040000e+02,0.0
50%,2017.000000,2018.000000,14.000000,6.800000,1.312000e+03,0.0
75%,2019.000000,2019.000000,30.000000,7.600000,6.166750e+03,0.0
max,2022.000000,2022.000000,8358.000000,9.700000,1.697849e+06,0.0


In [ ]:
df.isna().mean()

imdb_id          0.000000
title            0.000000
popular_rank     0.000000
certificate      0.656250
startYear        0.073487
endYear          0.839326
episodes         0.671233
runtime          0.000285
type             0.000285
orign_country    0.000000
language         0.000000
plot             0.056935
summary          0.000000
rating           0.143836
numVotes         0.143836
genres           0.000285
isAdult          0.000000
cast             0.000000
image_url        0.000000
dtype: float64

In [ ]:
df.shape

(7008, 19)

In [ ]:
red['title'][10]

'Dirty John'

In [ ]:
index = red.index
titles = red['title']
indices = pd.Series(index, index = titles).drop_duplicates()
indices

title
Lucifer                     0
Army of the Dead            1
The Kominsky Method         2
Friends                     3
Ragnarok                    4
                         ... 
 Merry Happy Whatever    7003
 The Morning Show        7004
 Losers                  7005
 Firefly Lane            7006
 Medical Police          7007
Length: 7008, dtype: int64

In [ ]:
#Print plot overviews of the first 5 movies.

df['summary'].head()

0    Lucifer Morningstar, bored from his sulking li...
1    With the abandoned, walled city of Las Vegas o...
2    Michael Douglas plays an actor who made it big...
3    Ross Geller, Rachel Green, Monica Geller, Joey...
4    In the small fictional town of Edda coming of ...
Name: summary, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['summary'] = df['summary'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['summary'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(7008, 29336)

In [ ]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[5000:5010]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['ciro',
 'cirque',
 'cissy',
 'cited',
 'cities',
 'citizen',
 'citizenry',
 'citizens',
 'citizenship',
 'citizenships']

In [ ]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim.shape

(7008, 7008)

In [ ]:
cosine_sim[1]

array([0.00299229, 1.        , 0.00985012, ..., 0.01151057, 0.        ,
       0.        ])

In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [ ]:
indices[:10]

title
Lucifer                0
Army of the Dead       1
The Kominsky Method    2
Friends                3
Ragnarok               4
StartUp                5
Grey's Anatomy         6
Sweet Tooth            7
The Blacklist          8
Jupiter's Legacy       9
dtype: int64

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [ ]:
get_recommendations('Ragnarok')

4893    Dragons: Rescue Riders: Huttsgalor Holiday
983                                         V-Wars
1230                                   Let It Snow
3448                           Paranormal Survivor
4020                                        Target
3831                            Only for One Night
3667                             Generation Iron 2
1532                               Jusqu'au déclin
859                                     The I-Land
3613                                     Followers
Name: title, dtype: object